## Implementation on Film Trust Dataset

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix
from scipy.stats.stats import pearsonr
#data user
df=pd.read_table('ratings.txt',sep=' ',names=['uid','iid','rating'])
data=df.values
df.head()


In [ ]:
#Rating Matrix/// 
df['uid'] = df['uid'].astype("category")
df['iid'] = df['iid'].astype("category")
reviews = csc_matrix((df['rating'].astype(float), 
                         (df['uid'].cat.codes,
                          df['iid'].cat.codes)))
reviews.shape
ratings_matrix = reviews.toarray()

## TCFACO: Trust-aware collaborative Þltering method based on ant colony optimization

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix
from scipy.stats.stats import pearsonr
#data user
df=pd.read_table('ratings.txt',sep=' ',names=['uid','iid','rating'])
data=df.values
df.head()

In [ ]:
#Rating Matrix

df['uid'] = df['uid'].astype("category")
df['iid'] = df['iid'].astype("category")
reviews = csc_matrix((df['rating'].astype(float), 
                         (df['uid'].cat.codes,
                          df['iid'].cat.codes)))
reviews.shape
ratings_matrix = reviews.toarray()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
#Inputs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_absolute_error
import networkx as nx
import sys
np.set_printoptions(threshold=sys.maxsize)
#Trust network
G = nx.read_weighted_edgelist('trust.txt', delimiter=',')
G.remove_nodes_from(nx.isolates(G));

In [ ]:
##define function
# find next node, define a function
def ChooseNextNode(sim,xj,current_node,Select_users,visitednodes):
    next_node=1
    TH=2*np.min(1/sim[:,xj])
    for i in range(len(Select_users)):
        for j in range(len(visitednodes)):
            if int(Select_users[i,0])!= int(visitednodes[j]):
                s=int(Select_users[i,0])
                if sim[s,xj]!= 0:
                   Pj=1/sim[s,xj]     
                   if Pj<TH:
                     if s !=current_node: 
                       next_node=s
    return next_node

In [ ]:
## define vars
num_users=len(df['uid'].unique())
dau=np.zeros(num_users)
nodes=G.nodes()
dis=[]
k=1
Similarity = np.ones(num_users)*-1
Tau= np.zeros(num_users)   
Wau= np.zeros(num_users) 
Select_users= np.zeros((num_users,4))
sim=np.random.random((num_users,num_users)) 
Q=1 #is a constant         
alpha=1;        # Phromone Exponential Weight
beta=1;         # Heuristic Exponential Weight
nA=3;
nIter=1;
ku=3;
xj=7
tau=np.zeros(nA)
r=np.zeros(nA)
estimated_r=np.zeros(nA)
next_node=0

In [ ]:
dmax=0
ListTrueRating=[]
listpredict=[]

#Target user
for Target_user in range(20):
    ## Step I: Selecting some users using rating and trust information
    UserId = int(data[Target_user,1])
    #1.Compute similarity values between the target user 𝑎 and the others using

    # 𝑆𝑈 =Select a set of users where there similarities to 𝑎 are higher than 
    for i in range(num_users):
            if  i!=UserId:
                a_r= ratings_matrix[UserId ,:]
                u_r= ratings_matrix[i ,:]
                cor=pearsonr(a_r,u_r)
                Similarity[i,]=cor[0]
                a=str(UserId) 
                u=str(i) 
                if a in nodes and u in nodes : 

                #eq(6)
                    a_r= ratings_matrix[UserId ,:]
                    u_r= ratings_matrix[i ,:]
                    cor=pearsonr(a_r,u_r)
                    Similarity[i,]=cor[0]

                #eq(5)
                    for g in nx.connected_component_subgraphs(G):
                        if len(dis)>90 and len(dis)<91:
                            dis.append(nx.average_shortest_path_length(g))
                            dmax=max(dis)      

                #eq(4)   
                    try:
                     dau[i]=nx.shortest_path_length(G,a,u) 
                     Tau[i]=(dmax-dau[i]+1)/dmax
                    except nx.NetworkXNoPath:
                     Tau[i]=0
                #eq(3) 
                if Similarity[i]+Tau[i] !=0 and Similarity[i]*Tau[i]!=0:
                   Wau[i]=(2*Similarity[i]*Tau[i])/ (Similarity[i]+Tau[i])
                elif Similarity[i]==0 and Tau[i]!=0:
                   Wau[i]= Tau[i]
                elif Similarity[i]!=0 and Tau[i]==0:
                   Wau[i]= Similarity[i]

                #selet users who their similarities to the target user is higher than Theta
                Theta=0.2
                if Wau[i]>Theta:
                   Select_users[i,0]=data[i,0]
                   Select_users[i,1]=data[i,1]
                   Select_users[i,2]=data[i,2]
                   Select_users[i,3]=Wau[i]

    SU = Select_users[[i for i, x in enumerate(Select_users) if x.any()]] 
    ## Step II: Selecting top-users with ant colony and computes the weight degree
    #create graph of select_users
    n=len(SU)
    rv= np.zeros(n)  
    A = [[0] * n] * n 
    A = np.array(A)
    for i in range(n):
      for j in range(n):
          A[i,j]=100*sim[i,j]
    A=A/100
    G2 = nx.from_numpy_matrix(np.array(A)) 
    color_map = []
    for node in G2:
        if node ==Target_user:
            color_map.append('red')
        else: color_map.append('blue')      
    ## Initialize pheromone values
    nIter=50
    users=[]
    pheromone=[]
    for i in range(nIter):
        # place ants randomly
        Q=1    
        for ant in range(nA):
            visitednodes= np.zeros(ku)     
            #find current node of ant
            current_node=np.random.randint(0,len(Select_users)) 
            visitednodes[0] = current_node
            for k in range(1,ku):
                # Choose the next unvisited user 𝑢 using (7)
                nextnode=ChooseNextNode(sim,xj,current_node,Select_users,visitednodes)
                current_node=nextnode
                visitednodes[k]= current_node
            #print(visitednodes)
            #Predict missing rating of active user using top using selected user
            for i in range(len(visitednodes)):
               sum1=0
               sum2=0
               if visitednodes[i]!=0:
                  Iu=len(Select_users) 
                  rv=data[int(visitednodes[i]),2]
                  #eq(11) 
                  for i in range(len(visitednodes)):
                      sum1=0
                      sum2=0
                      if visitednodes[i]!=0:
                         rv=Select_users[i,2] 
                         sum1=sum1+sim[i,xj]*rv
                         sum2=sum2+sim[i,xj]
                         estimated_r=sum1/sum2
                         ru=Select_users[xj,2]
                         #eq(12) 
                         fitness=np.abs(estimated_r-ru)/Iu 
            #eq(7)             
            p=np.mean(fitness)
            #eq(10) 
            tau[ant]=(1-p)*tau[ant]+Q
            #print(tau[ant])
            users.append(visitednodes)      
            pheromone.append(tau[ant]) 
    ##Step III: Prediction
    top_k_users=[]
    #Sort the users based on their pheromone values in decreasing order
    pheromone=np.asarray(pheromone)
    idx=pheromone.argsort()[::-1][:n]

    ##Choose the top-k users with highest pheromones as final user
    top=5
    for k in range(top):
        top_k_users.append(users[idx[k]])

    ##Predict unknown rating values for a  
    rating_user=[];
    for i in range(top):
         rating_user.append(data[int(np.asarray(top_k_users[i][0]))][2])

    rating_predict=np.mean(rating_user)   
    listpredict.append(rating_predict)
    rating_true=data[Target_user,2]
    ListTrueRating.append(rating_true)
  


In [ ]:
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(ListTrueRating, listpredict) 
print('MAE TCFACO=',MAE)
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(ListTrueRating, listpredict, squared=False)
print('RMSE TCFACO=',RMSE)